In [15]:
!git clone https://github.com/openai/grade-school-math.git

fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [16]:
!pip install transformers pandas jsonlines

In [17]:
from transformers import pipeline

# Load a pre-trained model for text generation
model = pipeline('text-generation', model='gpt2')


In [24]:
import jsonlines
import pandas as pd
from transformers import pipeline

# Load the dataset
data_path = 'grade-school-math/grade_school_math/data/train.jsonl'
data = []
with jsonlines.open(data_path) as reader:
    for obj in reader:
        data.append(obj)

# Convert to a DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [25]:
model = pipeline('text-generation', model='gpt2')

In [26]:
# Get the first problem from the dataset
problem = "If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?"

# Create a prompt
prompt = f"Solve the following math problem: {problem}"
print(prompt)

# Generate a response using the model
response = model(prompt, max_length=50)

# Print the response
print(response[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? It will stop. It will stop at a speed of 100 miles per hour. If it only stops at


3.2 Prompt Enginnering with Context

In [27]:
# Prompt Engineering with Context
context_problem = "If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? Provide the answer in miles."
context_prompt = f"Solve the following math problem with appropriate units: {context_problem}"
print("Context Prompt:")
print(context_prompt)

# Generate a response using the model
context_response = model(context_prompt, max_length=50, truncation=True)
print("Context Response:")
print(context_response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context Prompt:
Solve the following math problem with appropriate units: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? Provide the answer in miles.
Context Response:
Solve the following math problem with appropriate units: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? Provide the answer in miles. If the answer is 40 miles, then the answer is 0


3.3 Self Consistency

In [28]:
# Self-Consistency
self_consistent_problem = "If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?"
self_consistent_prompt = f"Solve the following math problem: {self_consistent_problem}"
print("Self-Consistent Prompt:")
print(self_consistent_prompt)

# Generate multiple responses using the model
responses = [model(self_consistent_prompt, max_length=50, truncation=True)[0]['generated_text'] for _ in range(5)]
print("Self-Consistent Responses:")
for i, response in enumerate(responses):
    print(f"Response {i + 1}: {response}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Self-Consistent Prompt:
Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Self-Consistent Responses:
Response 1: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? The answer is 3.5 to 3.7 miles per hour. If a car travels at 110 miles
Response 2: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? If it's at a speed of 60 miles per hour a minute, how long does it spend in its
Response 3: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?


Answer:


Take a car with at least 300 feet of straight sides

take two
Response 4: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? We find that on our test cars, the rate of travel is 20 miles per hour. But our test
Response 5: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? And

Zero Shot

In [31]:
# Zero-Shot Learning
zero_shot_problem = "If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?"
zero_shot_prompt = f"Solve the following math problem: {zero_shot_problem}"
print("Zero-Shot Prompt:")
print(zero_shot_prompt)

# Generate a response using the model
zero_shot_response = model(zero_shot_prompt, max_length=50, truncation=True)
print("Zero-Shot Response:")
print(zero_shot_response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zero-Shot Prompt:
Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?
Zero-Shot Response:
Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? Using two figures. The speeds to which your driving is determined based on this equation:

Where is


Chain of Thought

In [33]:
# Chain of Thought
chain_of_thought_problem = "If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?"
chain_of_thought_prompt = f"Let's solve the following math problem step-by-step:\n1. The car travels at a speed of 60 miles per hour.\n2. It travels for 3 hours.\n3. The distance traveled is speed multiplied by time.\nSolve the problem: {chain_of_thought_problem}"
print("Chain of Thought Prompt:")
print(chain_of_thought_prompt)

# Generate a response using the model
chain_of_thought_response = model(chain_of_thought_prompt, max_length=100, truncation=True)
print("Chain of Thought Response:")
print(chain_of_thought_response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chain of Thought Prompt:
Let's solve the following math problem step-by-step:
1. The car travels at a speed of 60 miles per hour.
2. It travels for 3 hours.
3. The distance traveled is speed multiplied by time.
Solve the problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?
Chain of Thought Response:
Let's solve the following math problem step-by-step:
1. The car travels at a speed of 60 miles per hour.
2. It travels for 3 hours.
3. The distance traveled is speed multiplied by time.
Solve the problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?
And how about a 100 mile trip?
And how about 2 hours and 30 minutes of travel time and 100


Zero Shot and Chain of thought

In [34]:
# Zero-Shot Chain of Thought
zero_shot_chain_of_thought_problem = "If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?"
zero_shot_chain_of_thought_prompt = f"Solve the following math problem step-by-step: {zero_shot_chain_of_thought_problem}"
print("Zero-Shot Chain of Thought Prompt:")
print(zero_shot_chain_of_thought_prompt)

# Generate a response using the model
zero_shot_chain_of_thought_response = model(zero_shot_chain_of_thought_prompt, max_length=100, truncation=True)
print("Zero-Shot Chain of Thought Response:")
print(zero_shot_chain_of_thought_response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zero-Shot Chain of Thought Prompt:
Solve the following math problem step-by-step: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?
Zero-Shot Chain of Thought Response:
Solve the following math problem step-by-step: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?

How much is the current daily amount of carbon dioxide in the air (COD) in the driving environment?

Is the current car's interior equipped with air suspension?

How much is required to change the climate to make the car in the car go on the wrong trail?

The following is a


Output comparison

In [36]:
# Function to evaluate and print responses
def evaluate_responses(responses, method_name):
    print(f"\n{method_name} Responses:")
    for i, response in enumerate(responses):
        print(f"Response {i + 1}: {response}")

# Evaluate Basic Prompting
basic_responses = [model(prompt, max_length=50, truncation=True)[0]['generated_text']]
evaluate_responses(responses, "Basic Prompting")

# Evaluate Prompt Engineering with Context
context_responses = [model(context_prompt, max_length=50, truncation=True)[0]['generated_text']]
evaluate_responses(context_responses, "Prompt Engineering with Context")

# Evaluate Self-Consistency
self_consistent_responses = [model(self_consistent_prompt, max_length=50, truncation=True)[0]['generated_text'] for _ in range(5)]
evaluate_responses(self_consistent_responses, "Self-Consistency")

# Evaluate Zero-Shot Learning
zero_shot_responses = [model(zero_shot_prompt, max_length=50, truncation=True)[0]['generated_text']]
evaluate_responses(zero_shot_responses, "Zero-Shot Learning")

# Evaluate Chain of Thought
chain_of_thought_responses = [model(chain_of_thought_prompt, max_length=100, truncation=True)[0]['generated_text']]
evaluate_responses(chain_of_thought_responses, "Chain of Thought")

# Evaluate Zero-Shot Chain of Thought
zero_shot_chain_of_thought_responses = [model(zero_shot_chain_of_thought_prompt, max_length=100, truncation=True)[0]['generated_text']]
evaluate_responses(zero_shot_chain_of_thought_responses, "Zero-Shot Chain of Thought")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Basic Prompting Responses:
Response 1: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? The answer is 3.5 to 3.7 miles per hour. If a car travels at 110 miles
Response 2: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? If it's at a speed of 60 miles per hour a minute, how long does it spend in its
Response 3: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?


Answer:


Take a car with at least 300 feet of straight sides

take two
Response 4: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? We find that on our test cars, the rate of travel is 20 miles per hour. But our test
Response 5: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? An

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Prompt Engineering with Context Responses:
Response 1: Solve the following math problem with appropriate units: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? Provide the answer in miles.

If a car travels at a speed of 60 miles


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Self-Consistency Responses:
Response 1: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? The answer: "6 miles!" We don't know. But we see the result of the "speed
Response 2: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? To solve this problem, I'm going to write the following equation:

The following has been repeated
Response 3: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? Does it take 4 hours to go 65 MPH? If so, why?

This question will ask
Response 4: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? We can solve this by dividing the speed of the car by the number of miles traveled (the speed limit
Response 5: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Zero-Shot Learning Responses:
Response 1: Solve the following math problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? How far does it travel? Are there things moving there? Or does it move randomly?

And


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Chain of Thought Responses:
Response 1: Let's solve the following math problem step-by-step:
1. The car travels at a speed of 60 miles per hour.
2. It travels for 3 hours.
3. The distance traveled is speed multiplied by time.
Solve the problem: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel?
6. There's a number of factors to consider:
How fast the car goes, how fast it's

Zero-Shot Chain of Thought Responses:
Response 1: Solve the following math problem step-by-step: If a car travels at a speed of 60 miles per hour for 3 hours, how far does it travel? If the car travels at 5 mph. You have 60 miles to travel or 30 mph to travel to. How far does it travel? What is the range from the car's speed to its destination? It gets about 3 miles. Do I have any idea where I am traveling from? This is a very simple math problem for solving


Analysis

In [41]:
import random

# Helper function to extract answers from the model response
def extract_answer(response_text):
    # This is a simple extraction based on the assumption the answer is numeric
    return ''.join(filter(str.isdigit, response_text))

# Function to evaluate accuracy
def evaluate_accuracy(generated_answers, correct_answers):
    correct = 0
    for gen, correct in zip(generated_answers, correct_answers):
        if extract_answer(gen) == correct:
            correct += 1
    return correct / len(correct_answers)

# Function to rate reasoning quality (requires manual inspection, placeholder function)
def rate_reasoning_quality(generated_answers):
    # Placeholder implementation
    ratings = []
    for answer in generated_answers:
        ratings.append(random.randint(1, 5))  # Random rating for demonstration
    return sum(ratings) / len(ratings)

# Function to calculate consistency
def evaluate_consistency(responses):
    # Placeholder implementation: Assuming responses are lists of answers
    return 0  # Placeholder: actual implementation requires analysis of consistency

# Select five random examples from the dataset
random_examples = df.sample(5).to_dict(orient='records')

# Correct answers (for evaluation purposes, we assume they are given in the dataset)
correct_answers = [example['answer'] for example in random_examples]

# Placeholder correct answers (for demonstration)
correct_answers = ['180', '240', '90', '300', '150']  # Replace with actual correct answers

# Generate responses using different methods
prompt_responses = [model(f"Solve the following math problem: {ex['question']}", max_length=170, truncation=True)[0]['generated_text'] for ex in random_examples]
context_responses = [model(f"Solve the following math problem with appropriate units: {ex['question']}", max_length=170, truncation=True)[0]['generated_text'] for ex in random_examples]
self_consistent_responses = [model(f"Solve the following math problem: {ex['question']}", max_length=170, truncation=True)[0]['generated_text'] for ex in random_examples for _ in range(5)]
zero_shot_responses = [model(f"Solve the following math problem: {ex['question']}", max_length=170, truncation=True)[0]['generated_text'] for ex in random_examples]
chain_of_thought_responses = [model(f"Let's solve the following math problem step-by-step:\n1. The car travels at a speed of 60 miles per hour.\n2. It travels for 3 hours.\n3. The distance traveled is speed multiplied by time.\nSolve the problem: {ex['question']}", max_length=100, truncation=True)[0]['generated_text'] for ex in random_examples]
zero_shot_chain_of_thought_responses = [model(f"Solve the following math problem step-by-step: {ex['question']}", max_length=100, truncation=True)[0]['generated_text'] for ex in random_examples]

# Evaluate accuracy
prompt_accuracy = evaluate_accuracy(prompt_responses, correct_answers)
context_accuracy = evaluate_accuracy(context_responses, correct_answers)
self_consistent_accuracy = evaluate_accuracy(self_consistent_responses, correct_answers)
zero_shot_accuracy = evaluate_accuracy(zero_shot_responses, correct_answers)
chain_of_thought_accuracy = evaluate_accuracy(chain_of_thought_responses, correct_answers)
zero_shot_chain_of_thought_accuracy = evaluate_accuracy(zero_shot_chain_of_thought_responses, correct_answers)

# Rate reasoning quality
prompt_reasoning_quality = rate_reasoning_quality(prompt_responses)
context_reasoning_quality = rate_reasoning_quality(context_responses)
self_consistent_reasoning_quality = rate_reasoning_quality(self_consistent_responses)
zero_shot_reasoning_quality = rate_reasoning_quality(zero_shot_responses)
chain_of_thought_reasoning_quality = rate_reasoning_quality(chain_of_thought_responses)
zero_shot_chain_of_thought_reasoning_quality = rate_reasoning_quality(zero_shot_chain_of_thought_responses)

# Evaluate consistency (requires proper implementation)
self_consistent_consistency = evaluate_consistency(self_consistent_responses)

# Print evaluation results
print("Evaluation Results:")
print(f"Prompting - Accuracy: {prompt_accuracy}, Reasoning Quality: {prompt_reasoning_quality}")
print(f"Prompt Engineering with Context - Accuracy: {context_accuracy}, Reasoning Quality: {context_reasoning_quality}")
print(f"Self-Consistency - Accuracy: {self_consistent_accuracy}, Reasoning Quality: {self_consistent_reasoning_quality}, Consistency: {self_consistent_consistency}")
print(f"Zero-Shot Learning - Accuracy: {zero_shot_accuracy}, Reasoning Quality: {zero_shot_reasoning_quality}")
print(f"Chain of Thought - Accuracy: {chain_of_thought_accuracy}, Reasoning Quality: {chain_of_thought_reasoning_quality}")
print(f"Zero-Shot Chain of Thought - Accuracy: {zero_shot_chain_of_thought_accuracy}, Reasoning Quality: {zero_shot_chain_of_thought_reasoning_quality}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

ValueError: Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

# Dataset
The GSM8K dataset contains math questions and answers designed to support the task of question answering on basic mathematical problems that require multi-step reasoning. Make sure you familiarize yourself with the dataset before starting the assignment.
Methods
Basic Prompting
For basic prompting, we crafted a simple prompt for the model to solve a GSM8K problem without any additional guidance. The prompt was structured as follows:
```python
basic_prompt = f"Solve the following math problem: {problem}"
response = model(basic_prompt, max_length=50, truncation=True)
```
Prompt Engineering with Context
In this method, we provided additional context to guide the model towards more accurate responses. We included units of measurement in the prompt:
```python
context_prompt = f"Solve the following math problem with appropriate units: {problem}"
response = model(context_prompt, max_length=50, truncation=True)
```
Self-Consistency
We generated multiple responses to the same prompt and chose the most consistent answer. The prompt was repeated five times for each problem:
```python
responses = [model(basic_prompt, max_length=50, truncation=True) for _ in range(5)]
```
Zero-Shot Learning
For zero-shot learning, we provided the model with a single example problem without any additional context or examples:
```python
zero_shot_prompt = f"Solve the following math problem: {problem}"
response = model(zero_shot_prompt, max_length=50, truncation=True)
```
Chain of Thought
This method encouraged the model to reason step-by-step by breaking down the problem into smaller steps:
```python
chain_of_thought_prompt = f"Let's solve the following math problem step-by-step:\n1. The car travels at a speed of 60 miles per hour.\n2. It travels for 3 hours.\n3. The distance traveled is speed multiplied by time.\nSolve the problem: {problem}"
response = model(chain_of_thought_prompt, max_length=100, truncation=True)
```
Zero-Shot Chain of Thought
This method used zero-shot prompts to encourage the model to think step-by-step:
```python
zero_shot_chain_of_thought_prompt = f"Solve the following math problem step-by-step: {problem}"
response = model(zero_shot_chain_of_thought_prompt, max_length=100, truncation=True)
```

# Discussion
Basic Prompting
**Pros**:
- Simple to implement.
- Requires minimal computational resources.

**Cons**:
- Often produces incomplete or incorrect answers.
- Lacks detailed reasoning.

Prompt Engineering with Context
**Pros**:
- Improves accuracy by providing additional context.
- Helps the model understand units of measurement.

**Cons**:
- Requires more effort to craft prompts.
- May still produce incorrect answers if context is not sufficient.

Self-Consistency
**Pros**:
- Increases the reliability of answers by averaging multiple responses.
- Reduces the impact of outlier responses.

**Cons**:
- Computationally expensive due to multiple generations.
- Consistency can vary depending on the model's randomness.

Zero-Shot Learning
**Pros**:
- Requires no additional examples.
- Easy to implement.

**Cons**:
- Lower accuracy compared to methods with examples or step-by-step reasoning.
- Limited ability to understand complex problems without context.

Chain of Thought
**Pros**:
- Encourages step-by-step reasoning, improving accuracy and reasoning quality.
- Helps the model break down complex problems.

**Cons**:
- Requires carefully crafted prompts.
- May be sensitive to the quality of the steps provided.

Zero-Shot Chain of Thought
**Pros**:
- Combines the benefits of zero-shot learning and chain-of-thought reasoning.
- Encourages the model to think step-by-step without examples.

**Cons**:
- May not always produce the desired step-by-step reasoning.
- Requires careful prompt crafting to be effective.
